In [7]:
import sys
sys.path.insert(0, "/home/user/Projects/OrangePi")

%load_ext autoreload
%autoreload 2

from orangepi.imports import *
from orangepi.__init__ import *
from orangepi.dataset_utils import split_dataset, DeepGlobeDataset
from orangepi.model_utils import *


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
transform = None
# Load model directly
from transformers import AutoImageProcessor, SegformerForSemanticSegmentation

processor = AutoImageProcessor.from_pretrained("nvidia/segformer-b1-finetuned-cityscapes-1024-1024", 
                                                size={"height": 512, "width": 512})
model = SegformerForSemanticSegmentation.from_pretrained("nvidia/segformer-b1-finetuned-cityscapes-1024-1024")

model.decode_head.classifier = nn.Conv2d(256, NUM_CLASSES, kernel_size=1)
torch.nn.init.xavier_uniform_(model.decode_head.classifier.weight)
torch.nn.init.zeros_(model.decode_head.classifier.bias)

model_type = "segformer"

In [3]:
# смотрим параметры модели
summary(
    model=model,
    input_size=(1, 3, 1024, 1024),
    col_names=["input_size", "output_size", "num_params", "trainable"],
    col_width=20,
    row_settings=["var_names"],
)

Layer (type (var_name))                                                     Input Shape          Output Shape         Param #              Trainable
SegformerForSemanticSegmentation (SegformerForSemanticSegmentation)         [1, 3, 1024, 1024]   [1, 1, 256, 256]     --                   True
├─SegformerModel (segformer)                                                [1, 3, 1024, 1024]   [1, 64, 256, 256]    --                   True
│    └─SegformerEncoder (encoder)                                           [1, 3, 1024, 1024]   [1, 64, 256, 256]    --                   True
│    │    └─ModuleList (patch_embeddings)                                   --                   --                   (recursive)          True
│    │    └─ModuleList (block)                                              --                   --                   (recursive)          True
│    │    └─ModuleList (layer_norm)                                         --                   --                   (recursive)  

In [9]:
dataset_path = Path("../data/DeepGlobeDataset/")
train_dir = dataset_path / "train"
valid_dir = dataset_path / "valid"
test_dir = dataset_path / "test"

train_dataset = DeepGlobeDataset(
    train_dir, model_type, transforms=transform, processor=processor, image_size=1024
)
valid_dataset = DeepGlobeDataset(
    valid_dir, model_type, transforms=transform, processor=processor, image_size=1024
)
test_dataset = DeepGlobeDataset(
    test_dir, model_type, transforms=transform, processor=processor, image_size=1024
)

train_loader = DataLoader(train_dataset, batch_size=2, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=2, shuffle=True)
test_loader  =  DataLoader(test_dataset, batch_size=2, shuffle=True)
len(train_loader), len(valid_loader), len(test_loader)

(2179, 312, 623)

In [1]:
epochs = 10
lr = 1e-3

# Запускаем обучение
model_save_name = "SegformerB0_DeepGlobe_V1"
assert model_save_name is not None, "Enter model's name for saving"

criterion = nn.BCEWithLogitsLoss()
best_model, loss_history, iou_score_history = train(
    model,
    train_loader,
    valid_loader,
    criterion,
    lr,
    epochs,
    model_type,
    verbose=True,
    is_scheduler=False,
)

# --- 6. Сохранение модели ---
torch.save(best_model.state_dict(), f"../models/{model_save_name}.pth")
print("Модель сохранена!")

NameError: name 'nn' is not defined

Recall: 0.8
